# GRAPE for preparing a cat state using L-BFGS Tutorial

<img src="../../_static/notebook-images/cat_adam_tut.png" alt="cat state" width="800"/>

In [1]:
# ruff: noqa
import os

os.sys.path.append("../../../..")

In [2]:
# ruff: noqa
import feedback_grape.grape as fg
from feedback_grape.utils.solver import sesolve
import jax.numpy as jnp
from feedback_grape.utils.operators import *
from feedback_grape.utils.states import basis, coherent
from feedback_grape.utils.tensor import tensor

## Step 1: initialize parameters

In [3]:
T = 1  # microsecond
num_of_intervals = 100
N = 30  # dimension of hilbert space
alpha = 1.5
# Phase for the interference
phi = jnp.pi

In [4]:
hconj = lambda a: jnp.swapaxes(a.conj(), -1, -2)

In [5]:
chi = 0.2385 * (2 * jnp.pi)
mu_qub = 4.0
mu_cav = 8.0

## Step 2: define initial and target states

In [6]:
psi0 = tensor(basis(2), basis(N))
cat_target_state = coherent(N, alpha) + jnp.exp(-1j * phi) * coherent(
    N, -alpha
)
psi_target = tensor(basis(2), cat_target_state)

## Step 3: Build the hamiltonian

In [7]:
# Using Jaynes-Cummings model for qubit + cavity
def build_grape_format_ham():
    """
    Build Hamiltonian for given (complex) e_qub and e_cav
    """

    a = tensor(identity(2), destroy(N))
    adag = tensor(identity(2), create(N))
    n_phot = adag @ a
    sigz = tensor(sigmaz(), identity(N))
    sigp = tensor(sigmap(), identity(N))
    one = tensor(identity(2), identity(N))

    H0 = +(chi / 2) * n_phot @ (sigz + one)
    H_ctrl_qub = mu_qub * sigp
    H_ctrl_qub_dag = hconj(H_ctrl_qub)
    H_ctrl_cav = mu_cav * adag
    H_ctrl_cav_dag = hconj(H_ctrl_cav)

    H_ctrl = [H_ctrl_qub, H_ctrl_qub_dag, H_ctrl_cav, H_ctrl_cav_dag]

    return H0, H_ctrl

## Step 4: Run GRAPE

In [8]:
# Outputs Fidelity of 0.9799029117042408 but in like 30 minutes
H0, H_ctrl = build_grape_format_ham()
res = fg.optimize_pulse(
    H0,
    H_ctrl,
    psi0,
    psi_target,
    num_t_slots=num_of_intervals,
    total_evo_time=T,
    evo_type="state",
    optimizer="l-bfgs",
    learning_rate=0.05,
    progress=True,
)
print("Final fidelity: ", res.final_fidelity)

Iteration 0, Loss: -0.002472
Iteration 10, Loss: -0.465461
Iteration 20, Loss: -0.492495
Iteration 30, Loss: -0.494823
Iteration 40, Loss: -0.494931
Iteration 50, Loss: -0.494940
Iteration 60, Loss: -0.494946
Iteration 70, Loss: -0.494956
Iteration 80, Loss: -0.494958
Iteration 90, Loss: -0.494959
Iteration 100, Loss: -0.496321
Iteration 110, Loss: -0.500120
Iteration 120, Loss: -0.505632
Iteration 130, Loss: -0.532456
Iteration 140, Loss: -0.563978
Iteration 150, Loss: -0.673139
Iteration 160, Loss: -0.684518
Iteration 170, Loss: -0.686454
Iteration 180, Loss: -0.690384
Iteration 190, Loss: -0.708800
Iteration 200, Loss: -0.712677
Iteration 210, Loss: -0.715881
Iteration 220, Loss: -0.740456
Iteration 230, Loss: -0.772951
Iteration 240, Loss: -0.807485
Iteration 250, Loss: -0.844914
Iteration 260, Loss: -0.873113
Iteration 270, Loss: -0.880960
Iteration 280, Loss: -0.890916
Iteration 290, Loss: -0.903942
Iteration 300, Loss: -0.908800
Iteration 310, Loss: -0.909721
Iteration 320, Loss

## Reconstructing hamiltonian from output signals

In [9]:
def build_ham_reconstructed(u1, u2, u3, u4):
    """
    Build Hamiltonian for given (complex) e_qub and e_cav
    """

    a = tensor(identity(2), destroy(N))
    adag = tensor(identity(2), create(N))
    n_phot = adag @ a
    sigz = tensor(sigmaz(), identity(N))
    sigp = tensor(sigmap(), identity(N))
    one = tensor(identity(2), identity(N))

    H0 = +(chi / 2) * n_phot @ (sigz + one)
    H_ctrl_qub = mu_qub * sigp
    H_ctrl_qub_dag = hconj(H_ctrl_qub)
    H_ctrl_cav = mu_cav * adag
    H_ctrl_cav_dag = hconj(H_ctrl_cav)

    # Apply control amplitudes
    H_ctrl = (
        u1 * H_ctrl_qub
        + u2 * H_ctrl_qub_dag
        + u3 * H_ctrl_cav
        + u4 * H_ctrl_cav_dag
    )

    H = H0 + H_ctrl
    return H

In [10]:
u1 = res.control_amplitudes[:, 0]
u2 = res.control_amplitudes[:, 1]
u3 = res.control_amplitudes[:, 2]
u4 = res.control_amplitudes[:, 3]

In [11]:
H_total = jnp.array(
    [
        build_ham_reconstructed(u1[i], u2[i], u3[i], u4[i])
        for i in range(len(u1))
    ]
)

In [12]:
time_start = 0.0
time_end = 1.0
# Eqivalant to delta_ts = jnp.repeat(0.2, time_intervals_num).astype(jnp.float32)
# However, it is implemented in this way to be more general and
# show that these are the differences between the time intervals
t_grid = jnp.linspace(time_start, time_end, num_of_intervals + 1)
delta_ts = t_grid[1:] - t_grid[:-1]

In [14]:
psi_fg = sesolve(H_total, psi0, delta_ts, evo_type="state")
print(fg.fidelity(C_target=psi_target, U_final=psi_fg, evo_type="state"))

0.928946810775257
